In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import os
import json
from tensorflow.python.keras.models import Model
from tensorflow.python.keras.layers import Dropout, Flatten, Dense
from tensorflow.python.keras import optimizers
# specifically for cnn
from tensorflow.python.keras.layers import Dropout, Flatten,Activation
from tensorflow.python.keras.layers import Conv2D, MaxPooling2D, BatchNormalization


C:\Users\User\anaconda3\envs\tf1\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\User\anaconda3\envs\tf1\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\User\anaconda3\envs\tf1\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\User\anaconda3\envs\tf1\lib\site-packages\tensorflow\python\framework\dty

In [ ]:
# train_dir = "C:/Users/Landy/Desktop/Angela/prodetection2/train/train/"
# test_dir = "C:/Users/Landy/Desktop/Angela/prodetection2/test/test/"
# train_df = pd.read_csv("C:/Users/Landy/Desktop/Angela/prodetection2/train.csv")
# train_df = train_df.sort_values(by='category',ascending=True)
# train_df.head()
pro_dir =  "C:/Users/User/Desktop/prodetection/"
train_dir = "C:/Users/User/Desktop/prodetection/train/train"
test_dir = "C:/Users/User/Desktop/prodetection/test/test"
train_df = pd.read_csv("C:/Users/User/Desktop/prodetection/train.csv")
train_df = train_df.sort_values(by='category',ascending=True)
train_df.head()

,filename,category
55585,9f89d9582e4d811330e83e3628b84114.jpg,0
53785,0e506b549c296564b55ed42283b87f64.jpg,0
53784,096f4541edcd6d8c209512f4a1a149a9.jpg,0
53783,89e70f1b22897e15782cda74fae117fc.jpg,0
53782,7f4c6b4f2177fa92d0d94b4ee2003ed3.jpg,0


In [ ]:
# IMG_SIZE = 32
IMG_SIZE = 256

In [ ]:
tf.VERSION

'1.14.0'

In [ ]:
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.10,
    shear_range=0.2,
    zoom_range=0.2,
)
# 1 2 4 7 8 14 16 28 56 112 941 1882 3764 6587 7528 13174 15056 26348 52696 
train_generator = train_datagen.flow_from_directory(
    directory = train_dir,
    target_size=(IMG_SIZE,IMG_SIZE),
    subset="training",
    batch_size=941,
    shuffle=True,
    class_mode="categorical"
)
#generator will return batch_x，batch_y
X, y = train_generator.next()

Found 94869 images belonging to 42 classes.


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.1, random_state=87)

In [ ]:
y[:2]

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)

In [ ]:
import h5py
data_name = "convert_img(%s x %s)" % (IMG_SIZE,IMG_SIZE)
labels_name = "labels(%s x %s)" % (IMG_SIZE,IMG_SIZE)

In [ ]:
# save converted data as hdf5
with h5py.File( pro_dir + data_name + '52696'+'.h5', 'w') as hf:
    hf.create_dataset(data_name,  data=X )
with h5py.File( pro_dir + labels_name + '52696'+'.h5', 'w') as hf:
    hf.create_dataset(labels_name,  data=y)
print("Successfully saved.")

In [ ]:
# X = []
# y = []
#read/write,file must exist
# load model2
# model2 = h5py.File( pro_dir +"model2/model2"+ '.h5', 'r')
# hfX = h5py.File( pro_dir + data_name +'.h5', 'r')
# hfy = h5py.File( pro_dir + labels_name +'.h5', 'r')
    
# X = np.array(hfX[data_name][:]) #dataset_name is same as hdf5 object name 
# y = np.array(hfy[labels_name][:])

In [ ]:
X.shape

(941, 256, 256, 3)

In [ ]:
# np.set_printoptions(threshold=np.inf)
y[:3]

(28, 42)

In [ ]:
# from tensorflow.python.keras.applications.efficientnet import EfficientNetB7
from efficientnet.tfkeras import EfficientNetB7

# include_top=False(是否包含頂部(Top) 『完全連接層』)
efnmodel = EfficientNetB7(weights='imagenet',include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3)) 

Instructions for updating:
`normal` is a deprecated alias for `truncated_normal`
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [ ]:
efnmodel.trainable = False
#efnmodel.summary()

In [ ]:
# add output layers and fine-tuning
x = efnmodel.output
x = Flatten()(x)
# 42 classes
x = Dense(42, activation="relu")(x)
# x = BatchNormalization()(x)
x = Dropout(0.2)(x)
predictions = Dense(42, activation="softmax")(x)
model = Model(efnmodel.input, predictions)
#sgd預設值
opt = tf.keras.optimizers.RMSprop(learning_rate=1e-3)
model.compile(optimizers=opt,loss='categorical_crossentropy',metrics=['categorical_accuracy'])
# model.summary

In [ ]:
# batch_size = int(len(train_df)/256) 1
batch_size = 1
nb_epoch = 2
# STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size

In [ ]:
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint
EARLY_STOP_PATIENCE = 3

cb_early_stopper = EarlyStopping(monitor = 'accuracy', patience = EARLY_STOP_PATIENCE)
cb_checkpointer = ModelCheckpoint(filepath = pro_dir+'model5/model5_best.h5', monitor = 'val_loss', save_best_only = True, mode = 'auto')

In [ ]:
# Train model
history = model.fit(
              X , y ,
              batch_size = batch_size,
              validation_data=(X_valid,y_valid),
              epochs = nb_epoch,
              shuffle = True,
              verbose = 1)

Train on 941 samples, validate on 95 samples
Epoch 1/2
941/941 [==============================] - 51s 55ms/sample - loss: 3.7366 - categorical_accuracy: 0.0308 - val_loss: 4.3988 - val_categorical_accuracy: 0.0000e+00
Epoch 2/2
941/941 [==============================] - 40s 42ms/sample - loss: 3.7328 - categorical_accuracy: 0.0351 - val_loss: 4.4602 - val_categorical_accuracy: 0.0211


In [ ]:
from tensorflow.python.keras.models import save_model
from tensorflow.python.keras.models import load_model

model.save(pro_dir+'model2/model3_batch64.h5')
# model=load_model(pro_dir+'model2/model2.h5')

In [ ]:
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
test_df = pd.read_csv(pro_dir+'test.csv')
# convert test to array
test_datagen = ImageDataGenerator(
    rescale=1./255
)
# 1 2 3 6 9 18 677 1354 2031 4062 6093
test_generator = test_datagen.flow_from_dataframe(
    dataframe = test_df,
    directory = test_dir,
    target_size=(IMG_SIZE,IMG_SIZE),
    x_col='filename',
    y_col=None,
    batch_size=1,
    shuffle=False,
    class_mode=None
)
#generator will return batch_x，batch_y
test_img  = test_generator.next()

Found 12186 validated image filenames.


In [ ]:
test_name = "test_img(%s x %s)" % (IMG_SIZE,IMG_SIZE)

with h5py.File( pro_dir +test_name + '.h5', 'w') as hf:
    hf.create_dataset(test_name,  data = test_img)

# could not broadcast input array from shape (3293,80,80,3) into shape (3293)

In [ ]:
test_generator.reset()
STEP_SIZE_TEST=test_generator.n//test_generator.batch_size

Y_pred = model.predict_generator(test_img,
                        steps=STEP_SIZE_TEST,
                        verbose=1)

12186/12186 [==============================] - 314s 26ms/step


In [ ]:
predicted_class_indices=np.argmax(Y_pred,axis=1)
predicted_class_indices[:3]

array([0, 8, 0], dtype=int64)

In [ ]:
labels = (train_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]
# for i in range(0, len(predictions)): 
#     predictions[i] = int(predictions[i]) 
prediction = np.array(predictions,dtype=np.int64)

In [ ]:
filenames=test_generator.filenames
results=pd.DataFrame({"filename":filenames,
                      "category":predictions})
results['category'] = results['category'].astype(np.int64)
results.to_csv(pro_dir+"results.csv",index=False)

In [ ]:
# for i in range(1,len(train_df)):
#     if len(train_df)%int(i)==0:
#         print(i,end=' ')
results['category'][1].type
#記得把0換成00 以此類推

AttributeError: 'numpy.int64' object has no attribute 'type'